In [1]:
import urllib.request
import pandas as pd
import requests
from bs4 import BeautifulSoup
import gzip
import os
from time import sleep

## 2020-2023

In [33]:
def county_processing(county):
    suffix = [' County', ' Census Area',  ' Municipality', ' City and Borough',' Borough', 
              ' Parish', ' city', ' Co.', ' C.A.', ' Bor.', ' Par.', ' Co', ' Par']
    for s in suffix:
        county = county.removesuffix(s)
    return county

def processing_csv(df, year_begin, year_end):
    def FIPS(st, c):
        st, c = str(st), str(c)
        if len(st)==1:
            st = '0'+st 
        while len(c)<3:
            c = '0'+c 
        return st+c 
    df = df[df['STNAME']!=df['CTYNAME']]
    df['fips'] =  df.apply(lambda row: FIPS(row['STATE'], row['COUNTY'] ), axis = 1)
    columns = ['fips'] + ['STNAME', 'CTYNAME'] + [f'POPESTIMATE{str(x)}' for x in range(year_begin,year_end+1)]
    df = df[columns]
    
    df.columns = ['fips'] + ['state', 'county'] + [str(x) for x in range(year_begin,year_end+1)]
    df['county'] = df['county'].apply(county_processing)
    df_processed = pd.melt(df, 
                            id_vars=['fips', 'county', 'state'], 
                            value_vars = [str(x) for x in range(year_begin,year_end+1)], 
                            var_name = 'year', 
                            value_name = 'population')
    return df_processed


In [34]:
df1 = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2020-2023/counties/totals/co-est2023-alldata.csv', 
                  encoding ='ISO-8859-1')

df1['STATE'].isnull().sum()

df1_processed = processing_csv(df1, 2020, 2023)
df1_processed.head()

/var/folders/qc/t2cn0cp92hg4klpsfc6zft9r0000gn/T/ipykernel_2592/3550782135.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fips'] =  df.apply(lambda row: FIPS(row['STATE'], row['COUNTY'] ), axis = 1)


,fips,county,state,year,population
0,01001,Autauga,Alabama,2020,58915
1,01003,Baldwin,Alabama,2020,233227
2,01005,Barbour,Alabama,2020,24969
3,01007,Bibb,Alabama,2020,22188
4,01009,Blount,Alabama,2020,59107


## 2010-2019

In [35]:
df2 = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv', 
                  encoding ='ISO-8859-1')

In [39]:
df2_processed = processing_csv(df2, 2010, 2019)


/var/folders/qc/t2cn0cp92hg4klpsfc6zft9r0000gn/T/ipykernel_2592/3550782135.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fips'] =  df.apply(lambda row: FIPS(row['STATE'], row['COUNTY'] ), axis = 1)


## 2000-2009

In [40]:
df3 = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2000-2009/counties/totals/co-est2009-alldata.csv', 
                  encoding ='ISO-8859-1')

In [41]:
df3_processed = processing_csv(df3, 2000, 2009)

/var/folders/qc/t2cn0cp92hg4klpsfc6zft9r0000gn/T/ipykernel_2592/3550782135.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fips'] =  df.apply(lambda row: FIPS(row['STATE'], row['COUNTY'] ), axis = 1)


In [42]:
df3_processed.head()

,fips,county,state,year,population
0,01001,Autauga,Alabama,2000,43872
1,01003,Baldwin,Alabama,2000,141358
2,01005,Barbour,Alabama,2000,29035
3,01007,Bibb,Alabama,2000,19936
4,01009,Blount,Alabama,2000,51181


## 1990-1999

In [44]:
response = requests.get(url="https://www2.census.gov/programs-surveys/popest/tables/1990-2000/estimates-and-change-1990-2000/2000c8_00.txt")
soup = BeautifulSoup(response.content, 'html.parser')
text = soup.get_text()

In [46]:
text_list = text.split('\n')
text_list = [L for L in text_list if (len(L)>0 and L[0] == '1' and L[1] != '0')][1:]
text_list = [L.split() for L in text_list]

text_list

[['1',
  '01',
  '4,385,739',
  '4,369,224',
  '4,350,594',
  '4,320,692',
  '4,290,678',
  '4,263,125',
  '4,233,214',
  '4,193,639',
  '4,139,880',
  '4,091,397',
  '4,048,526',
  '4,040,389',
  'Alabama'],
 ['1',
  '01001',
  '43,981',
  '43,021',
  '42,156',
  '41,280',
  '40,241',
  '39,140',
  '38,208',
  '36,969',
  '35,995',
  '35,016',
  '34,354',
  '34,222',
  'Autauga',
  'County'],
 ['1',
  '01003',
  '139,254',
  '135,671',
  '132,798',
  '128,760',
  '124,243',
  '119,947',
  '115,823',
  '110,872',
  '106,234',
  '102,228',
  '98,918',
  '98,280',
  'Baldwin',
  'County'],
 ['1',
  '01005',
  '26,706',
  '26,713',
  '26,914',
  '26,739',
  '26,853',
  '26,654',
  '26,779',
  '26,634',
  '26,437',
  '26,229',
  '25,452',
  '25,417',
  'Barbour',
  'County'],
 ['1',
  '01007',
  '19,785',
  '19,551',
  '18,980',
  '18,638',
  '18,253',
  '17,985',
  '17,640',
  '17,445',
  '17,191',
  '16,955',
  '16,674',
  '16,598',
  'Bibb',
  'County'],
 ['1',
  '01009',
  '47,994',
  

In [47]:
def extract_name(L):
    return ' '.join([x for x in L if not x[0].isdigit()])
state_code = dict([(L[1], extract_name(L)) for L in text_list if len(L[1])==2])

In [57]:
county_list = [  [L[1]] + [county_processing(extract_name(L)), state_code[L[1][:2]]] + [int(x.replace(',','')) for x in L[3:13]] for L in text_list if len(L[1])!=2]

In [58]:
df4 = pd.DataFrame( county_list, columns = ['fips', 'county', 'state'] + [str(1999-x) for x in range(10)])

In [61]:
df4.head()

,fips,county,state,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,01001,Autauga,Alabama,43021,42156,41280,40241,39140,38208,36969,35995,35016,34354
1,01003,Baldwin,Alabama,135671,132798,128760,124243,119947,115823,110872,106234,102228,98918
2,01005,Barbour,Alabama,26713,26914,26739,26853,26654,26779,26634,26437,26229,25452
3,01007,Bibb,Alabama,19551,18980,18638,18253,17985,17640,17445,17191,16955,16674
4,01009,Blount,Alabama,47308,46276,44889,43538,42581,41360,40990,40228,39864,39409


In [62]:
df4_processed = pd.melt(df4, id_vars=['fips', 'county','state'], value_vars = [str(x) for x in range(1990,2000)], 
                            var_name = 'year', 
                            value_name = 'population')
df4_processed.head()

,fips,county,state,year,population
0,01001,Autauga,Alabama,1990,34354
1,01003,Baldwin,Alabama,1990,98918
2,01005,Barbour,Alabama,1990,25452
3,01007,Bibb,Alabama,1990,16674
4,01009,Blount,Alabama,1990,39409


## 1980-1990


In [80]:
response = requests.get(url="https://www2.census.gov/programs-surveys/popest/tables/1980-1990/counties/totals/e8089co.txt")
soup = BeautifulSoup(response.content, 'html.parser')
text = soup.get_text()

text_list = text.split('\n')
text_list = text_list[23:]
text_list = [L.split() for L in text_list]

i = 0

while i<len(text_list):
    try:
        if len(text_list[i])<=5:
            text_list[i] = text_list[i] + text_list[i+1]
            del text_list[i+1]
        i+=1
    except:
        break




sub_tables = []
for L in text_list:
    if len(L) == 0:
        pass
    elif L[0] == 'FIPS':
        sub_tables.append([])
    else:
        if L[0] != '00000':
            sub_tables[-1].append(L)

for i in range(len(sub_tables)):
    if i%2 == 1:
        sub_tables[i] = [L[-5:] for L in sub_tables[i]]


                 

In [81]:
correct_sub_tables = []
for i in range(51):
    n = len(sub_tables[2*i])
    correct_sub_tables.append([ sub_tables[2*i][j] + sub_tables[2*i+1][j] for j in range(n)])
correct_sub_tables = [L[1:] for L in correct_sub_tables] #remove header
states = [L[0][1] for L in correct_sub_tables] #extract state name
correct_sub_tables = [L[1:] for L in correct_sub_tables] #remove row of state data
for i in range(51):
    for j in range(len(correct_sub_tables[i])):
        L = correct_sub_tables[i][j]
        county = county_processing(' '.join(L[1:-10]))
        L.append(L[0])
        L.append(county)
        L.append(state_code[L[0][:2]])
        correct_sub_tables[i][j] = L[-3:]+ L[-13:-3]
        
correct_sub_tables = [x for xs in correct_sub_tables for x in xs]



In [82]:
df5 = pd.DataFrame( correct_sub_tables, columns = ['fips', 'county', 'state'] + [str(x) for x in range(1980, 1990)])
df5_processed = pd.melt(df5, id_vars=['fips', 'county','state'], value_vars = [str(x) for x in range(1980,1990)], 
                            var_name = 'year', 
                            value_name = 'population')

In [84]:
df5_processed.sample(10)

,fips,county,state,year,population
25261,05053,Grant,Arkansas,1988,13794
7622,27063,Jackson,Minnesota,1982,13485
14523,37147,Pitt,North Carolina,1984,96164
30253,37197,Yadkin,North Carolina,1989,30349
13816,26053,Gogebic,Michigan,1984,19628
1860,36073,Orleans,New York,1980,38496
22134,05081,Little River,Arkansas,1987,13786
13812,26045,Eaton,Michigan,1984,87868
23882,37019,Brunswick,North Carolina,1987,48428
18032,45031,Darlington,South Carolina,1985,62152


## 1970-1979

In [87]:
response = requests.get(url="https://www2.census.gov/programs-surveys/popest/tables/1900-1980/counties/totals/e7079co.txt")
soup = BeautifulSoup(response.content, 'html.parser')
text = soup.get_text()

text_list = text.split('\n')
text_list = text_list[26:]
text_list = [L.split() for L in text_list]

i = 0

while i<len(text_list):
    try:
        if len(text_list[i])<=5:
            text_list[i] = text_list[i] + text_list[i+1]
            del text_list[i+1]
        i+=1
    except:
        break




sub_tables = []
for L in text_list:
    if len(L) == 0:
        pass
    elif L[0] == 'FIPS':
        sub_tables.append([])
    else:
        if L[0] != '00000':
            sub_tables[-1].append(L)

for i in range(len(sub_tables)):
    if i%2 == 1:
        sub_tables[i] = [L[-5:] for L in sub_tables[i]]

correct_sub_tables = []
for i in range(51):
    n = len(sub_tables[2*i])
    correct_sub_tables.append([ sub_tables[2*i][j] + sub_tables[2*i+1][j] for j in range(n)])
correct_sub_tables = [L[1:] for L in correct_sub_tables] #remove header
correct_sub_tables = [L[1:] for L in correct_sub_tables] #remove row of state data

for i in range(51):
    for j in range(len(correct_sub_tables[i])):
        L = correct_sub_tables[i][j]
        county = county_processing(' '.join(L[1:-10]))
        L.append(L[0])
        L.append(county)
        L.append(state_code[ L[0][:2] ])
        correct_sub_tables[i][j] = L[-3:]+ L[-13:-3]
        
correct_sub_tables = [x for xs in correct_sub_tables for x in xs]

df6 = pd.DataFrame( correct_sub_tables, columns = ['fips', 'county', 'state'] + [str(x) for x in range(1970, 1980)])
df6.head()
df6_processed = pd.melt(df6, id_vars=['fips', 'county','state'], value_vars = [str(x) for x in range(1970,1980)], 
                            var_name = 'year', 
                            value_name = 'population')

In [88]:
df6_processed[df6_processed['state'] == 'Louisiana']

,fips,county,state,year,population
1105,22001,Acadia,Louisiana,1970,52109
1106,22003,Allen,Louisiana,1970,20794
1107,22005,Ascension,Louisiana,1970,37086
1108,22007,Assumption,Louisiana,1970,19654
1109,22009,Avoyelles,Louisiana,1970,37751
...,...,...,...,...,...
29406,22119,Webster,Louisiana,1979,43500
29407,22121,West Baton Rouge,Louisiana,1979,18800
29408,22123,West Carroll,Louisiana,1979,13000
29409,22125,West Feliciana,Louisiana,1979,11500


# Final processing


In [89]:
df_final = pd.concat([df1_processed,df2_processed,df3_processed,df4_processed,df5_processed,df6_processed])

In [90]:
df_final

,fips,county,state,year,population
0,01001,Autauga,Alabama,2020,58915
1,01003,Baldwin,Alabama,2020,233227
2,01005,Barbour,Alabama,2020,24969
3,01007,Bibb,Alabama,2020,22188
4,01009,Blount,Alabama,2020,59107
...,...,...,...,...,...
31375,56037,Sweetwater,Wyoming,1979,39600
31376,56039,Teton,Wyoming,1979,9500
31377,56041,Uinta,Wyoming,1979,11800
31378,56043,Washakie,Wyoming,1979,9100


In [120]:
df_coordinate = pd.read_csv('county_coordinate.csv', dtype = {'fips': str })
df_coordinate = df_coordinate[df_coordinate.columns[1:]]
df_coordinate.head()

,fips,state,county,latitude,longitude
0,01001,Alabama,Autauga,32.536,-86.644
1,01003,Alabama,Baldwin,30.659,-87.746
2,01005,Alabama,Barbour,31.871,-85.405
3,01007,Alabama,Bibb,33.016,-87.127
4,01009,Alabama,Blount,33.977,-86.567


### Merging via name

In [629]:
def ad_hoc_county(county):
    mapper = {'LaSalle': 'La Salle', 'La Porte': 'LaPorte', 'DeBaca': 'De Baca', 
              'De Kalb': 'DeKalb', 'Matanuska- Susitna':'Matanuska-Susitna', 'Kauai':'Kauaʻi',
            'Yellowstone National Park':'Yellowstone', 'Kusilvak': 'Wade Hampton', 'Hawaii': 'Hawaiʻi',
            'D.C.': 'District of Columbia', 'Lagrange':'LaGrange', 'Doña Ana':'Dona Ana',
            'Wade Hampton C.A':'Wade Hampton', 'HillsBor.':'Hillsborough', 'Natchitoches Par':'Natchitoches'
              }
    # Shannon in south dakota should be Oglala Lakota, but Shannon in Missouri is still Shannon
    if county in mapper:
        return mapper[county]
    else:
        return county

df_final['county'] = df_final['county'].apply(ad_hoc_county)

def ad_hoc(county,state):
    mapper = { ('Shannon', 'South Dakota') : ('Oglala Lakota', 'South Dakota'), ('Dade', 'Florida'): ('Miami-Dade', 'Florida') }
    if (county,state) in mapper:
        return mapper[(county,state)]
    else:
        return county,state

df_final[['county','state']] = df_final.apply(lambda row: ad_hoc(row['county'], row['state']), axis=1, result_type='expand')

df = pd.merge(df_final, df_coordinate, how='left', on=['county','state'])
df['population'] = df['population'].astype('float32').astype('int32')


In [630]:
df[(df['latitude'].isna())].sample(15)

,county,state,year,population,latitude,longitude
154651,Aleutian Islands,Alaska,1975,8500,NaN,NaN
160692,South Boston,Virginia,1976,6800,NaN,NaN
132600,Skagway-Yakutat- Angoon C.A,Alaska,1988,4112,NaN,NaN
157540,South Boston,Virginia,1975,6800,NaN,NaN
129449,Wrangell- Petersburg C.A,Alaska,1987,6953,NaN,NaN
154666,Prin Wales- Outer Ketch,Alaska,1975,4200,NaN,NaN
154356,Clifton Forge,Virginia,1974,5300,NaN,NaN
85274,P. of Wales-Outer Ketchikan(1),Alaska,1993,6705,NaN,NaN
91590,Wrangell-Petersburg,Alaska,1995,7019,NaN,NaN
126008,South Boston,Virginia,1985,7078,NaN,NaN


In [635]:
print(f'Number of rows with missing coordinates: {len(df[df["latitude"].isna()] ) }')

Number of rows with missing coordinates: 158


In [633]:
df[df['latitude'].isna()].sort_values(by=['population'], ascending=False).head(60)

,county,state,year,population,latitude,longitude
157803,Aleutian Islands,Alaska,1976,8600,NaN,NaN
160955,Aleutian Islands,Alaska,1977,8500,NaN,NaN
154651,Aleutian Islands,Alaska,1975,8500,NaN,NaN
151499,Aleutian Islands,Alaska,1974,8500,NaN,NaN
138891,Aleutian Islands,Alaska,1970,8221,NaN,NaN
164107,Aleutian Islands,Alaska,1978,8100,NaN,NaN
142043,Aleutian Islands,Alaska,1971,8000,NaN,NaN
167259,Aleutian Islands,Alaska,1979,7800,NaN,NaN
145195,Aleutian Islands,Alaska,1972,7800,NaN,NaN
148347,Aleutian Islands,Alaska,1973,7700,NaN,NaN


### Merging via FIPS

In [121]:
df_fips = pd.merge(df_final, df_coordinate, how='left', on= ['fips'], suffixes=('','_y'))
df_fips = df_fips[['fips', 'county', 'state', 'year', 'population', 'latitude', 'longitude']]
df_fips['population'] = df_fips['population'].astype('float32').astype('int32')


In [123]:
len(df_fips[df_fips['latitude'].isna()].sort_values(by=['population'], ascending=False))

172

In [124]:
df_fips.to_csv("county_population.csv")